In [1]:
import pandas as pd
import gradio as gr


def get_BFCs(results_df):
    df = results_df[['id', 'commit_hash', 'is_bug_fixing_commit']]
    df = df.rename(columns={'commit_hash': 'hash', 'is_bug_fixing_commit': 'BFC'})
    df['hash'] = df['hash'].str.slice(0, 10)
    return df

results_A = pd.read_csv('130_results_michel.csv')
results_B = pd.read_csv('130_results_abhishek.csv')
results_C = pd.read_csv('130_results_david.csv')

BFCs_A = get_BFCs(results_A)
BFCs_B = get_BFCs(results_B)
BFCs_C = get_BFCs(results_C)

BFCs = pd.merge(BFCs_A, BFCs_B, on='hash', how='inner', suffixes=('A', 'B'))

BFCs = pd.merge(BFCs, BFCs_C, on='hash', how='inner', suffixes=('', 'C'))
BFCs = BFCs[['hash', 'BFCA', 'BFCB', 'BFC']]
BFCs = BFCs.rename(columns={'BFC': 'BFCC'})

/home/jgb/src/papers/linuxbugs/linux-bugs/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [128]:
# Manage reviewer file
review_cols = ['hash', 'reviewer', 'bfc', 'comment']
review_filename = '130_review.csv'
def load_review_df(filename):
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        df = pd.DataFrame(columns=review_cols)
    return df

def update_review_df(data):
    if any(BFCs_R['hash'] == data['hash']):
        BFCs_R.loc[BFCs_R['hash'] == data['hash'], data.keys()] = data.values()
    else:
        BFCs_R.loc[len(BFCs_R)] = data

def get_review_df(hash):
    result_df = BFCs_R[BFCs_R['hash'] == hash]
    if len(result_df) == 0:
        results = None
    else:
        results = result_df.iloc[0].to_dict()
    print(results)
    return results
        
def save_df(filename, df):
    df.to_csv(filename)

BFCs_R = load_review_df(review_filename)

In [118]:
def get_annotation(df, hash):
    """Produce markdown for the annotation of a given short hash
    in the annotation dataframe
    """
    rows = df.loc[df['commit_hash'].str.startswith(hash)]
    row = rows.iloc[0].to_dict()
    hash_long = row['commit_hash']
    link = f"https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/commit/?h=v6.7&id={hash}"
    is_bfc = row['is_bug_fixing_commit']
    is_obvious = row['is_obvious_bug']
    is_safety = row['is_safety_related']
    comment = row['comment']
    visited = row['link_visited']
    result = (f"[{hash}]({link})\n* bfc: {is_bfc}\n* obvious: {is_obvious}\n"
              f"* safety: {is_safety}\n* visited: {visited}\n"
              f"* comment: {comment}")
    return result
port =10020

In [130]:
with gr.Blocks() as ui:
    with gr.Row():
        filter_ddw = gr.Dropdown(label="Select commits",
                                 choices=[("All", "all"),
                                  ("All annotators equal", "all-equal"),
                                  ("Any annotator different", "any-dif"),
                                  ("Annotators A, B different", "ab-dif"),
                                  ("Annotators B, C different", "bc-dif"),
                                  ("Annotators A, C different", "ac-dif")],
                                value="all")
        search_txt = gr.Textbox(label="Search commit")
        count_md = gr.Markdown(f"Commits: {len(BFCs.index)}")
    bfcs_df = gr.Dataframe(value=BFCs, height=300)
    with gr.Row():
        A_md = gr.Markdown()
        B_md = gr.Markdown()
        C_md = gr.Markdown()
    with gr.Row():
        R_txt = gr.Textbox(label="Comment by reviewer",
                          lines=5, interactive=False)
        with gr.Column():
            hash_txt = gr.Textbox(label="Selected hash", interactive=False)
            Rbfc_dd = gr.Dropdown(label="Bug fixing commit",
                                 choices=[("True", True), ("False", False)],
                                 interactive=False)
            R_btn = gr.Button("Save review", interactive=False)

    @filter_ddw.change(inputs=[filter_ddw], outputs=[bfcs_df, count_md])
    def filter_records(choice):
        if choice == "all":
            df = BFCs
        elif choice == "all-equal":
            df = BFCs.query("(BFCA == BFCB) and (BFCB == BFCC)")
        elif choice == "any-dif":
            df = BFCs.query("(BFCA != BFCB) or (BFCB != BFCC)")
        elif choice == "ab-dif":
            df = BFCs.query("BFCA != BFCB")
        elif choice == "bc-dif":
            df = BFCs.query("BFCB != BFCC")
        elif choice == "ac-dif":
            df = BFCs.query("BFCA != BFCC")
        count = len(df.index)
        return df, f"Commits: {count}"

    @search_txt.change(inputs=[search_txt], outputs=[bfcs_df, count_md])
    def find_record(hash):
        df = BFCs[BFCs['hash'].str.startswith(hash)]
        count = len(df.index)
        return df, f"Commits: {count}"

    def select_commit(event: gr.SelectData, bfcs):
        #print(event.index, event.target, event.value, bfcs)
        row = bfcs.iloc[event.index[0]]
        hash = row['hash']
        annot_A = get_annotation(results_A, hash)
        annot_B = get_annotation(results_B, hash)
        annot_C = get_annotation(results_C, hash)
        review = get_review_df(hash)
        if review is None:
            review_text = ""
            review_bfc = None
        else:
            review_text = review['comment']
            review_bfc = review['bfc']
        return (hash,
            "**A**\n\n" + annot_A, "**B**\n\n" + annot_B, "**C**\n\n" + annot_C,
                gr.update(interactive=True, value=review_text),
                gr.update(interactive=True, value=review_bfc))
        
    bfcs_df.select(select_commit, [bfcs_df], [hash_txt, A_md, B_md, C_md,
                                              R_txt, Rbfc_dd])

    @R_btn.click(inputs=[hash_txt, R_txt, Rbfc_dd], outputs=[R_btn])
    def update_review(hash, review, bfc):
        update_review_df({'hash': hash,
                       'reviewer': 'jesus',
                       'bfc': bfc,
                       'comment': review})
        save_df(review_filename, BFCs_R)
        return gr.update(interactive=False)

    def review_changed():
        return gr.update(interactive=True)
        
    R_txt.change(review_changed, inputs=[], outputs=[R_btn])
    Rbfc_dd.change(review_changed, inputs=[], outputs=[R_btn])
    
#port = port + 1
#ui.launch(server_port=port)
ui.launch()

Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


In [135]:
BFCs_R

,hash,reviewer,bfc,comment
0,494603e06b,jesus,False,Comments
1,9b648bb1d8,jesus,True,Otro
